# 01: Quickstart — Define, Check, Break, Fix

This notebook walks through KindScript's core loop and all three constraint types:

| Constraint | Error Code | What it catches |
|----------|------------|----------------|
| `noDependency` | KS70001 | Forbidden imports between layers |
| `purity` | KS70003 | I/O imports in pure layers |
| `noCycles` | KS70004 | Circular dependencies between layers |

Each section: define the architecture, see the violation, fix it.

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
import { ksc, copyFixture, writeFile, readFile, cleanup } from './lib.ts';
console.log("Setup complete.");

---

## Part 1: `noDependency` — Forbidden Imports (KS70001)

The most common constraint. Forbids imports from one layer to another.

**Use case:** In Clean Architecture, the domain layer must not import from infrastructure.

In [ ]:
const demo1 = copyFixture("quickstart");
console.log("Demo project:", demo1);

In [ ]:
writeFile(demo1, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type DomainLayer = Kind<"DomainLayer">;
type ApplicationLayer = Kind<"ApplicationLayer">;
type InfrastructureLayer = Kind<"InfrastructureLayer">;

type CleanArchitectureContext = Kind<"CleanArchitectureContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [["domain", "infrastructure"], ["domain", "application"]];
}>;

export const app = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<CleanArchitectureContext, '.'>;
`);

console.log("Wrote src/context.ts — domain cannot import from infrastructure or application");

**`src/context.ts`** defines:
1. **Kind types** — `DomainLayer`, `ApplicationLayer`, `InfrastructureLayer` are leaf Kinds. `CleanArchitectureContext` groups them as members.
2. **Constraints** — `noDependency` forbids imports from domain to infrastructure and domain to application.
3. **Instance** — `satisfies Instance<T, '.'>` declares this as an instance of the Kind type. The second parameter `'.'` is the location path, relative to the declaration file — so `'.'` means `src/` (the directory containing `context.ts`). Member names map to subdirectories: `domain` → `src/domain/`, `infrastructure` → `src/infrastructure/`.

All APIs are pure types — zero runtime footprint. No config file needed.

In [ ]:
console.log("=== Check: should pass (no violations) ===");
await ksc("check", demo1);

In [ ]:
// Break it: domain imports from infrastructure
writeFile(demo1, "src/domain/user.ts", `
import { saveUser } from '../infrastructure/user-repo';

export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  const user = { id: Math.random().toString(36), name, email };
  saveUser(user); // BAD: domain reaching into infrastructure
  return user;
}
`);

console.log("=== Violation: domain imports from infrastructure ===");
await ksc("check", demo1);

KindScript caught it: `KS70001` — forbidden dependency. The error shows the file, the forbidden import path, and which contract caught it.

**Fix pattern:** Dependency injection. Define interfaces in the inner layer; implement them in the outer.

In [ ]:
// Fix: remove the infrastructure import, keep domain pure
writeFile(demo1, "src/domain/user.ts", `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`);

console.log("=== Fixed: domain has no infrastructure imports ===");
await ksc("check", demo1);

cleanup(demo1);

---

## Part 2: `purity` — No I/O in Pure Layers (KS70003)

Ensures a layer has no side effects — no `fs`, `http`, `net`, `child_process`, or any of Node's ~50 built-in I/O modules.

**Use case:** Domain logic should be pure. If it needs to read a file, it receives the data through a port.

Purity is declared as an **intrinsic constraint** on the leaf Kind:

```typescript
type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
```

When a composite Kind contains a member of this type, purity is automatically enforced.

In [ ]:
const demo2 = copyFixture("purity");

writeFile(demo2, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;

type AppContext = Kind<"AppContext", {
  domain: DomainLayer;
}>;

export const app = {
  domain: {},
} satisfies Instance<AppContext, '.'>;
`);

console.log("=== Violation: domain imports 'fs' ===");
await ksc("check", demo2);

In [ ]:
// Fix: inject data instead of reading directly
writeFile(demo2, "src/domain/service.ts", `
export interface DataReader {
  read(path: string): string;
}

export class DomainService {
  constructor(private reader: DataReader) {}
  readData(): string {
    return this.reader.read('/tmp/data.txt');
  }
}
`);

console.log("=== Fixed: domain uses injected reader ===");
await ksc("check", demo2);

cleanup(demo2);

---

## Part 3: `noCycles` — No Circular Dependencies (KS70004)

Detects circular dependency chains between layers.

```typescript
noCycles: ["domain", "infrastructure"]
```

In [ ]:
const demo3 = copyFixture("no-cycles");

writeFile(demo3, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type DomainLayer = Kind<"DomainLayer">;
type InfrastructureLayer = Kind<"InfrastructureLayer">;

type AppContext = Kind<"AppContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  noCycles: ["domain", "infrastructure"];
}>;

export const app = {
  domain: {},
  infrastructure: {},
} satisfies Instance<AppContext, '.'>;
`);

console.log("=== Violation: domain <-> infrastructure cycle ===");
await ksc("check", demo3);

Note the error format: `[domain]` instead of a file path. Cycles are structural (project-wide), so the diagnostic uses scope-based location.

In [ ]:
// Fix: break the cycle — use dependency inversion
writeFile(demo3, "src/domain/service.ts", `
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getData(): string[] { return this.store.query('SELECT *'); }
}
`);

writeFile(demo3, "src/infrastructure/database.ts", `
import { DataStore } from '../domain/service';

export class Database implements DataStore {
  query(sql: string): string[] { return [sql]; }
}
`);

console.log("=== Fixed: one-directional dependency (infra -> domain) ===");
await ksc("check", demo3);

cleanup(demo3);

---

## Summary

| Constraint | Code | Catches | Fix pattern |
|----------|------|---------|-------------|
| `noDependency` | KS70001 | Forbidden imports between layers | Dependency injection — define interfaces in the inner layer |
| `purity` | KS70003 | I/O imports (`fs`, `http`, etc.) in pure layers | Inject I/O through constructor |
| `noCycles` | KS70004 | Circular dependency chains | Break the cycle with interfaces (Dependency Inversion) |

Constraints are declared on the Kind type's third parameter. Purity can be declared as an intrinsic on the member Kind:

```typescript
type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;

type CleanContext = Kind<"CleanContext", {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [["domain", "infrastructure"], ["domain", "application"]];
  noCycles: ["domain", "application", "infrastructure"];
}>;

export const app = {
  domain: {},
  infrastructure: {},
} satisfies Instance<CleanContext, '.'>;
```

**Next:** See [02-real-world.ipynb](02-real-world.ipynb) for multi-instance bounded contexts, or [docs/06-tutorial.md](../docs/06-tutorial.md) for a static walkthrough.